In [67]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
from preprocessing import full_preprocessing

In [76]:
X_train, Y_train, Y_train_oh, ids, X_test = full_preprocessing()

In [77]:
# JE TESTE MA FONCTION full_preprocessing ICI

print(X_train[0:4], "\n")
print(Y_train[0:4], "\n")
print(Y_train_oh[0:4], "\n")
print(ids[0:4], "\n")
print(X_test[0:4], "\n")

print(type(Y_train_oh))
print(type(Y_train))
print(type(X_train))

['i m missing so much time off school but omg i ve never been so ill just wan na get better now', 'user no thanks for the follow', 'the chucky collection child s play child s play bride of chucky dvd url', 'user oh no that means that weather radio may be going off a lot and scaring the crap out of us'] 

[0 1 0 0] 

[[1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]] 

['1', '2', '3', '4'] 

['sea doo pro sea scooter sports with the portable seascootersave air stay longer in the water and url', 'user shucks well i work all week so now i ca not come cheer you on oh and put those batteries in your calculator', 'i cant stay away from bug thats my baby', 'user no lol im perfectly fine and not contagious anymore lmao'] 

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'list'>


In [47]:
#============================================================================
#============================================================================
####################### LA JE FAIS STEP BY STEP #############################
#============================================================================
#============================================================================

from preprocessing import *
from our_functionsv3 import convert_to_one_hot, get_test_data

path_pos = "twitter-datasets/train_pos.txt"
path_neg = "twitter-datasets/train_neg.txt"
path_test = "twitter-datasets/test_data.txt"

train_pos = read_data(path_pos)
train_neg = read_data(path_neg)

pos = pd.DataFrame(train_pos, columns=["tweet"])
neg = pd.DataFrame(train_neg, columns=["tweet"])

test = read_data(path_test)
test_pd = pd.DataFrame(test, columns=["tweet"])

train_pos_preprocessed = pre_process_tweets(pos)
train_neg_preprocessed = pre_process_tweets(neg)

################# SECOND PART ############

X_pos = list(train_pos_preprocessed.tweet)
X_neg = list(train_neg_preprocessed.tweet)
X_train = X_pos + X_neg

Y_pos = np.ones(len(X_pos), dtype = int)
Y_neg = np.zeros(len(X_neg), dtype = int)
Y_train = np.concatenate((Y_pos, Y_neg), axis = -1)

X_train, Y_train = shuffle(X_train, Y_train, random_state=52) #shuffle our training set
Y_train_oh = convert_to_one_hot(Y_train, C=2)

ids, _ = get_test_data(path_test)


test_preprocessed = pre_process_tweets(test_pd)
X_test = list(test_preprocessed.tweet)


In [49]:
######################## JE CHECK SI MA FONCTION A L'AIR OK !!!
X_train[0:4]
Y_train[0:4]
Y_train_oh[0:4]
X_test[0:8]

['sea doo pro sea scooter sports with the portable seascootersave air stay longer in the water and url',
 'user shucks well i work all week so now i ca not come cheer you on oh and put those batteries in your calculator',
 'i cant stay away from bug thats my baby',
 'user no lol im perfectly fine and not contagious anymore lmao',
 'whenever i fall asleep watching the tv i always wake up with a headache',
 'user he needs to get rid of that thing it scares me lol but he do not need a car either he needs drivers ed again',
 'its whatever in a terrible mood',
 'yesss rt user user thanks jordan i love you and i m gon na call you later']

In [16]:
type(train_pos_preprocessed)
train_pos_preprocessed.tweet[0:4]
type(train_pos_preprocessed.tweet[0])

str

In [26]:
#################################### LINK BETWEEN PART 1 AND 2 ##################################
X_pos_try = list(train_pos_preprocessed.tweet)
X_pos_try[0:8]

['user i dunno justin read my mention or not only justin and god knows about that but i hope you will follow me #believe',
 'because your logic is so dumb i wo not even crop out your name or your photo tsk url',
 'user just put casper in a box looved the battle #crakkbitch',
 'user user thanks sir do not trip lil mama just keep doin ya thang',
 'visiting my brother tmr is the bestest birthday gift eveerrr',
 'user yay #lifecompleted tweet facebook me to let me know please',
 'user #1dnextalbumtitle feel for you rollercoaster of life song cocept life #yolo becoming famous #followmeplz',
 'workin hard or hardly workin rt user at hardee s with my future coworker user']

In [10]:
################################### SECOND PART #####################################
from our_functionsv3 import read_data
from sklearn.utils import shuffle

pos = "twitter-datasets/train_pos_tokenize_not_hash.txt"
neg = "twitter-datasets/train_neg_tokenize_not_hash.txt"

test_path = "twitter-datasets/test_preprocessed_tokenize_not_hash.txt"

X_pos = read_data(pos)
X_neg = read_data(neg)
X_train = X_pos + X_neg

Y_pos = np.ones(len(X_pos), dtype = int)
Y_neg = np.zeros(len(X_neg), dtype = int)
Y_train = np.concatenate((Y_pos, Y_neg), axis = -1)

X_train_shuffled, Y_train_shuffled = shuffle(X_train, Y_train, random_state=52)

In [27]:
X_pos[0:4]

['user i dunno justin read my mention or not only justin and god knows about that but i hope you will follow me #believe',
 'because your logic is so dumb i wo not even crop out your name or your photo tsk url',
 'user just put casper in a box looved the battle #crakkbitch',
 'user user thanks sir do not trip lil mama just keep doin ya thang']